In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Quick start with Model Garden - MedSigLIP

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fgoogle-health%2Fmedsiglip%2Fmain%2Fnotebooks%2Fquick_start_with_model_garden.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/google-health/medsiglip/blob/main/notebooks/quick_start_with_model_garden.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates how to use MedSigLIP in Vertex AI to generate embeddings from medical images or text using two methods for getting predictions:

* **Online predictions** are synchronous requests that are made to the endpoint deployed from Model Garden and are served with low latency. Online predictions are useful if the embeddings are being used in production. The cost for online prediction is based on the time a virtual machine spends waiting in an active state (an endpoint with a deployed model) to handle prediction requests.

* **Batch predictions** are asynchronous requests that are run on a set number of images or text specified in a single job. They are made directly to an uploaded model and do not use an endpoint deployed from Model Garden. Batch predictions are useful if you want to generate embeddings for a large number of inputs for use in training and don't require low latency. The cost for batch prediction is based on the time a virtual machine spends running your prediction job.

Vertex AI makes it easy to serve your model and make it accessible to the world. Learn more about [Vertex AI](https://cloud.google.com/vertex-ai/docs/start/introduction-unified-platform).

### Objectives

- Deploy MedSigLIP to a Vertex AI Endpoint and get online predictions.
- Upload MedSigLIP to Vertex AI Model Registry and get batch predictions.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Before you begin

In [ ]:
# @title Import packages and define common functions

import datetime
import importlib
import io
import json
import os
import uuid

import numpy as np
from google.cloud import aiplatform, storage
from google.oauth2 import credentials
from PIL import Image

if not os.path.isdir("vertex-ai-samples"):
    ! git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git

common_util = importlib.import_module(
    "vertex-ai-samples.community-content.vertex_model_garden.model_oss.notebook_util.common_util"
)

models, endpoints = {}, {}


def download_gcs_image_bytes(gcs_uri, creds=None):
    """Download an image from Cloud Storage."""
    if creds:
        storage_client = storage.Client(credentials=creds)
    else:
        storage_client = storage.Client.create_anonymous_client()
    blob = storage.blob.Blob.from_string(gcs_uri, client=storage_client)
    return blob.download_as_bytes()

In [ ]:
# @title Set up Google Cloud environment

# @markdown #### Prerequisites

# @markdown 1. Make sure that [billing is enabled](https://cloud.google.com/billing/docs/how-to/modify-project) for your project.

# @markdown 2. Make sure that either the Compute Engine API is enabled or that you have the [Service Usage Admin](https://cloud.google.com/iam/docs/understanding-roles#serviceusage.serviceUsageAdmin) (`roles/serviceusage.serviceUsageAdmin`) role to enable the API.

# @markdown This section sets the default Google Cloud project and region, enables the Compute Engine API (if not already enabled), and initializes the Vertex AI API.

# Get the default project ID.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the Compute Engine API, if not already.
print("Enabling Compute Engine API.")
! gcloud services enable compute.googleapis.com

# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION)

## Get online predictions

In [ ]:
# @title Import deployed model

# @markdown To get [online predictions](https://cloud.google.com/vertex-ai/docs/predictions/get-online-predictions), you will need a MedSigLIP [Vertex AI Endpoint](https://cloud.google.com/vertex-ai/docs/general/deployment) that has been deployed from Model Garden. If you have not already done so, go to the [MedSigLIP model card](https://console.cloud.google.com/vertex-ai/publishers/google/model-garden/medsiglip) in Model Garden and click "Deploy options > Vertex AI" to deploy the model.

# @markdown This section gets the Vertex AI Endpoint resource that you deployed from Model Garden to use for online predictions.

# @markdown Fill in the endpoint ID and region below. You can find your deployed endpoint on the [Vertex AI online prediction page](https://console.cloud.google.com/vertex-ai/online-prediction/endpoints).

ENDPOINT_ID = ""  # @param {type: "string", placeholder:"e.g. 123456789"}
ENDPOINT_REGION = ""  # @param {type: "string", placeholder:"e.g. us-central1"}

# @markdown Set `use_dedicated_endpoint` if you are using a [dedicated endpoint](https://cloud.google.com/vertex-ai/docs/predictions/choose-endpoint-type) (`True` by default for Model Garden deployments). Uncheck this option for all other endpoint types.

use_dedicated_endpoint = True  # @param {type: "boolean"}

endpoints["endpoint"] = aiplatform.Endpoint(
    endpoint_name=ENDPOINT_ID,
    project=PROJECT_ID,
    location=ENDPOINT_REGION,
)

### Predict

You can send [online prediction](https://cloud.google.com/vertex-ai/docs/predictions/get-online-predictions) requests to the endpoint with medical images or text to generate embeddings.

The following examples demonstrate using MedSigLIP to generate:

* Image embeddings from
  * An image file stored in [Cloud Storage](https://cloud.google.com/storage/docs/introduction)
  * A base64-encoded string of the raw image bytes
* Text embeddings from a text query

In [ ]:
# @title #### (Optional) Resize images

# @markdown If you want to reproduce the results from MedSigLIP evals, we
# @markdown recommend a resizing operation with `tf.image.resize` to match the
# @markdown implementation with the Big Vision [library](https://github.com/google-research/big_vision/blob/0127fb6b337ee2a27bf4e54dea79cff176527356/big_vision/pp/ops_image.py#L84).

# @markdown Otherwise, you can rely on the Transformers image processor's
# @markdown built-in resizing (done automatically by default and uses
# @markdown `PIL.Image.resize`) or use another resizing method.

from tensorflow.image import resize as tf_resize
import numpy as np

INPUT_DIMENSION = [448, 448]


def resize(image):
    return Image.fromarray(
        tf_resize(
            images=image, size=INPUT_DIMENSION, method='bilinear', antialias=False
        ).numpy().astype(np.uint8)
    )

In [ ]:
# @title #### Generate an image embedding from a Cloud Storage image file

# @markdown This section shows an example of generating an embedding using an image from the [SCIN Dataset](https://github.com/google-research-datasets/scin) that is stored in Cloud Storage.

# @markdown The prediction request instance contains an `image` field with the following fields:
# @markdown - `gcs_uri`: `gs://` URI specifying the location of an image file stored in Cloud Storage
# @markdown - `access_credential`: Credentials used to access data in Cloud Storage (optional for public buckets)

# @markdown You can replace `GCS_URI` below to use your own data.

# @markdown Click "Show Code" to see more details.

GCS_URI = "gs://dx-scin-public-data/dataset/images/-1010754336982699838.png"  # @param {type:"string", placeholder:"Cloud Storage file URI"}

img = Image.open(io.BytesIO(download_gcs_image_bytes(GCS_URI)))
display(img)

instances = [
    {
        "image": {
            "gcs_uri": GCS_URI,
        },
    },
]

response = endpoints["endpoint"].predict(instances=instances, use_dedicated_endpoint=use_dedicated_endpoint)
predictions = response.predictions

img_embedding = np.array(predictions[0]["embedding"])
print("Embedding shape: ", img_embedding.shape)

In [ ]:
# @title #### Generate an image embedding from raw bytes

# @markdown This section shows an example of generating an embedding using a chest X-ray image that is downloaded from Wikimedia Commons as image bytes.

# @markdown The prediction request instance contains an `image` field with the following field:
# @markdown - `input_bytes`: Base-64 encoded string of the raw image bytes

# @markdown **Note:** The request must be within the Vertex AI endpoint [payload size limits](https://cloud.google.com/vertex-ai/docs/predictions/choose-endpoint-type).

# @markdown Click "Show Code" to see more details.

# Image attribution: Stillwaterising, CC0, via Wikimedia Commons
image_url = "https://upload.wikimedia.org/wikipedia/commons/c/c8/Chest_Xray_PA_3-8-2010.png"
! wget -nc -q {image_url}
filename = os.path.basename(image_url)
img = Image.open(filename)
display(img)

# Optionally resize image with tf.image.resize
img = resize(img)

instances = [
    {
        "image": {
            "input_bytes": common_util.image_to_base64(img, "PNG"),
        },
    },
]

response = endpoints["endpoint"].predict(
    instances=instances, use_dedicated_endpoint=use_dedicated_endpoint
)
predictions = response.predictions

img_embedding = np.array(predictions[0]["embedding"])

print("Embedding shape: ", img_embedding.shape)

In [ ]:
# @title ### Generate a text embedding from a text query

# @markdown This section shows an example of generating an embedding from a text query.

# @markdown The prediction request instance contains the following field:
# @markdown - `text`: Text query string

# @markdown Click "Show Code" to see more details.

TEXT_QUERY = "A chest X-ray with pneumonia."  # @param {type:"string", placeholder:"Text query string"}

instances = [{"text": TEXT_QUERY}]

response = endpoints["endpoint"].predict(
    instances=instances, use_dedicated_endpoint=use_dedicated_endpoint
)
predictions = response.predictions

txt_embedding = np.array(predictions[0]["embedding"])

print("Embedding shape: ", txt_embedding.shape)

## Get batch predictions

In [ ]:
# @title Upload model to Vertex AI Model Registry

# @markdown To get [batch predictions](https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions), you must first upload the prebuilt MedSigLIP model to [Vertex AI Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction). Batch prediction requests are made directly to a model in Model Registry without deploying to an endpoint.

MODEL_ID = "medsiglip-448"
MODEL_ARTIFACT_URI = f"gs://vertex-model-garden-restricted-us/medsiglip/{MODEL_ID}"

# The pre-built serving docker image.
SERVE_DOCKER_URI = "us-docker.pkg.dev/deeplearning-platform-release/vertex-model-garden/health-ai-medsiglip.1-0:model-garden.health-ai-medsiglip-release_20250709.00_p0"


def upload_model(model_name: str, artifact_uri: str) -> aiplatform.Model:
    env_vars = {
        "MODEL_INPUT_WIDTH": "448",
        "MODEL_INPUT_HEIGHT": "448",
        "TEXT_EMBEDDINGS_PER_BATCH_PREDICTION": "1",
        "IMAGE_EMBEDDINGS_PER_BATCH_PREDICTION": "1",
        "ENABLE_CLOUD_LOGGING": "false",
    }
    model = aiplatform.Model.upload(
        display_name=model_name,
        artifact_uri=artifact_uri,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[8080],
        serving_container_predict_route="/predict",
        serving_container_health_route="/health",
        serving_container_environment_variables=env_vars,
    )
    return model


models["model"] = upload_model(
    model_name=common_util.get_job_name_with_datetime(prefix=MODEL_ID),
    artifact_uri=MODEL_ARTIFACT_URI,
)

In [ ]:
# @title Set up Google Cloud resources

# @markdown This section sets up a [Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing batch prediction inputs and outputs and gets the [Compute Engine default service account](https://cloud.google.com/compute/docs/access/service-accounts#default_service_account) which will be used to run the batch prediction jobs.

# @markdown 1. Make sure that you have the following required roles:
# @markdown - [Storage Admin](https://cloud.google.com/iam/docs/understanding-roles#storage.admin) (`roles/storage.admin`) to create and use Cloud Storage buckets
# @markdown - [Service Account User](https://cloud.google.com/iam/docs/understanding-roles#iam.serviceAccountUser) (`roles/iam.serviceAccountUser`) on either the project or the Compute Engine default service account

# @markdown 2. Set up a Cloud Storage bucket.
# @markdown - A new bucket will automatically be created for you.
# @markdown - [Optional] To use an existing bucket, specify the `gs://` bucket URI. The specified Cloud Storage bucket should be located in the same region as where the notebook was launched. Note that a multi-region bucket (e.g. "us") is not considered a match for a single region (e.g. "us-central1") covered by the multi-region range.

BUCKET_URI = ""  # @param {type:"string", placeholder:"[Optional] Cloud Storage bucket URI"}

# Cloud Storage bucket for storing batch prediction artifacts.
# A unique bucket will be created for the purpose of this notebook. If you
# prefer using your own GCS bucket, change the value of BUCKET_URI above.
if BUCKET_URI is None or BUCKET_URI.strip() == "":
    now = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}-{str(uuid.uuid4())[:4]}"
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
    ! gcloud storage buckets create --location {REGION} {BUCKET_URI}
else:
    assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
    shell_output = ! gcloud storage buckets describe {BUCKET_NAME} | grep "location:" | sed "s/location://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            f"Bucket region {bucket_region} is different from notebook region {REGION}"
        )
print(f"Using this Cloud Storage Bucket: {BUCKET_URI}")

# Service account used for running the prediction container.
# Gets the Compute Engine default service account. If you prefer using your own
# custom service account, change the value of SERVICE_ACCOUNT below.
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print("Using this service account:", SERVICE_ACCOUNT)

### Predict

You can send [batch prediction requests](https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions#request_a_batch_prediction) to the model using a [JSON Lines](https://jsonlines.org/) file to specify a list of input instances with medical images or text to generate embeddings. For more details on configuring batch prediction jobs, see how to [format your input data](https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions#input_data_requirements) and [choose compute settings](https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions#choose_machine_type_and_replica_count).

The following examples demonstrate using MedSigLIP to generate:

* Image embeddings in batch from
  * Image files stored in [Cloud Storage](https://cloud.google.com/storage/docs/introduction)
  * Base64-encoded strings of the raw image bytes
* Text embeddings in batch from text queries

In [ ]:
# @title #### Generate image embeddings in batch from Cloud Storage image files

# @markdown This section shows an example of generating embeddings in batch using images from the [SCIN Dataset](https://github.com/google-research-datasets/scin) that is stored in Cloud Storage.

# @markdown Each line in the input JSON Lines file is a prediction request instance that contains an `image` field with the following field:
# @markdown - `gcs_uri`: `gs://` URI specifying the location of an image file stored in Cloud Storage
# @markdown - `access_credential`: Credentials used to access data in Cloud Storage, set to `"application_default"` for batch predictions

# @markdown You can replace `GCS_URIS` below use your own data.

# @markdown **Note:** The custom service account used to launch the batch prediction job must have permission to read the data from Cloud Storage.

# @markdown Click "Show Code" to see more details.

# Comma-separated list of Cloud Storage URIs
GCS_URIS = "gs://dx-scin-public-data/dataset/images/-1013653802423257870.png,gs://dx-scin-public-data/dataset/images/-106355051564979815.png"  # @param {type:"string", placeholder:"Comma-separated list of Cloud Storage file URIs"}

gcs_uris_list = GCS_URIS.split(",")
batch_predict_instances = [
    {
        "image": {
            "gcs_uri": uri,
            "access_credential": "application_default"
        }
    }
    for uri in gcs_uris_list
]

# Write instances to JSON Lines file
os.makedirs("batch_predict_input", exist_ok=True)
instances_filename = "gcs_instances.jsonl"
with open(f"batch_predict_input/{instances_filename}", "w") as f:
    for line in batch_predict_instances:
        json_str = json.dumps(line)
        f.write(json_str)
        f.write("\n")

# Copy the file to Cloud Storage
batch_predict_prefix = f"batch-predict-{MODEL_ID}"
! gcloud storage cp ./batch_predict_input/{instances_filename} {BUCKET_URI}/{batch_predict_prefix}/input/{instances_filename}

batch_predict_job_name = common_util.get_job_name_with_datetime(
    prefix=f"batch-predict-{MODEL_ID}"
)

gcs_batch_predict_job = models["model"].batch_predict(
    job_display_name=batch_predict_job_name,
    gcs_source=os.path.join(
        BUCKET_URI, batch_predict_prefix, f"input/{instances_filename}"
    ),
    gcs_destination_prefix=os.path.join(BUCKET_URI, batch_predict_prefix, "output"),
    machine_type="n1-standard-8",
    accelerator_type="NVIDIA_TESLA_T4",
    accelerator_count=1,
    service_account=SERVICE_ACCOUNT,
)

gcs_batch_predict_job.wait()

print(gcs_batch_predict_job.display_name)
print(gcs_batch_predict_job.resource_name)
print(gcs_batch_predict_job.state)

In [ ]:
# @title #### Generate image embeddings in batch from raw bytes

# @markdown This section shows an example of generating embeddings in batch using an image from the [SCIN Dataset](https://github.com/google-research-datasets/scin) that is downloaded as image bytes.

# @markdown Each line in the input JSON Lines file is a prediction request instance that contains an `image` field with the following field:
# @markdown - `input_bytes`: Base-64 encoded string of the raw image bytes

# @markdown Click "Show Code" to see more details.

image_urls = [
    "https://upload.wikimedia.org/wikipedia/commons/c/c8/Chest_Xray_PA_3-8-2010.png"
]
batch_predict_instances = []
for image_url in image_urls:
    ! wget -nc -q {image_url}
    filename = os.path.basename(image_url)
    img = Image.open(filename)
    batch_predict_instances.append(
        {"image": {"input_bytes": common_util.image_to_base64(img, "PNG")}}
    )

# Write instances to JSON Lines file
os.makedirs("batch_predict_input", exist_ok=True)
instances_filename = "bytes_instances.jsonl"
with open(f"batch_predict_input/{instances_filename}", "w") as f:
    for line in batch_predict_instances:
        json_str = json.dumps(line)
        f.write(json_str)
        f.write("\n")

# Copy the file to Cloud Storage
batch_predict_prefix = f"batch-predict-{MODEL_ID}"
! gcloud storage cp ./batch_predict_input/{instances_filename} {BUCKET_URI}/{batch_predict_prefix}/input/{instances_filename}

batch_predict_job_name = common_util.get_job_name_with_datetime(
    prefix=f"batch-predict-{MODEL_ID}"
)

bytes_batch_predict_job = models["model"].batch_predict(
    job_display_name=batch_predict_job_name,
    gcs_source=os.path.join(
        BUCKET_URI, batch_predict_prefix, f"input/{instances_filename}"
    ),
    gcs_destination_prefix=os.path.join(BUCKET_URI, batch_predict_prefix, "output"),
    machine_type="n1-standard-8",
    accelerator_type="NVIDIA_TESLA_T4",
    accelerator_count=1,
    service_account=SERVICE_ACCOUNT,
)

bytes_batch_predict_job.wait()

print(bytes_batch_predict_job.display_name)
print(bytes_batch_predict_job.resource_name)
print(bytes_batch_predict_job.state)

In [ ]:
# @title #### Generate text embeddings in batch from text queries

# @markdown This section shows an example of generating embeddings in batch from text queries.

# @markdown Each line in the input JSON Lines file is a prediction request instance that contains the following field:
# @markdown - `text`: Text query string

# @markdown Click "Show Code" to see more details.

TEXT_QUERIES = "A chest X-ray with pneumonia."  # @param {type:"string", placeholder:"Comma-separated list of text query strings"}

text_queries_list = TEXT_QUERIES.split(",")
batch_predict_instances = [{"text": text} for text in text_queries_list]

# Write instances to JSON Lines file
os.makedirs("batch_predict_input", exist_ok=True)
instances_filename = "text_instances.jsonl"
with open(f"batch_predict_input/{instances_filename}", "w") as f:
    for line in batch_predict_instances:
        json_str = json.dumps(line)
        f.write(json_str)
        f.write("\n")

# Copy the file to Cloud Storage
batch_predict_prefix = f"batch-predict-{MODEL_ID}"
! gcloud storage cp ./batch_predict_input/{instances_filename} {BUCKET_URI}/{batch_predict_prefix}/input/{instances_filename}

batch_predict_job_name = common_util.get_job_name_with_datetime(
    prefix=f"batch-predict-{MODEL_ID}"
)

text_batch_predict_job = models["model"].batch_predict(
    job_display_name=batch_predict_job_name,
    gcs_source=os.path.join(
        BUCKET_URI, batch_predict_prefix, f"input/{instances_filename}"
    ),
    gcs_destination_prefix=os.path.join(BUCKET_URI, batch_predict_prefix, "output"),
    machine_type="n1-standard-8",
    accelerator_type="NVIDIA_TESLA_T4",
    accelerator_count=1,
    service_account=SERVICE_ACCOUNT,
)

text_batch_predict_job.wait()

print(text_batch_predict_job.display_name)
print(text_batch_predict_job.resource_name)
print(text_batch_predict_job.state)

In [ ]:
# @title #### Get prediction results

# @markdown This section shows an example of [retrieving batch prediction results](https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions#retrieve_batch_prediction_results) from the JSON Lines file(s) in the output Cloud Storage location.

# @markdown Click "Show Code" to see more details.


def download_gcs_files_as_json(gcs_files_prefix):
    """Download specified files from Cloud Storage and convert content to JSON."""
    lines = []
    client = storage.Client()
    bucket = storage.bucket.Bucket.from_string(BUCKET_NAME, client)
    blobs = bucket.list_blobs(prefix=gcs_files_prefix)
    for blob in blobs:
        with blob.open("r") as f:
            for line in f:
                lines.append(json.loads(line))
    return lines


# Get results from the first batch prediction job (with Cloud Storage inputs)
# You can replace this variable to get results from another batch prediction job
batch_predict_job = gcs_batch_predict_job
batch_predict_output_dir = batch_predict_job.output_info.gcs_output_directory
batch_predict_output_files_prefix = os.path.join(
    batch_predict_output_dir.replace(f"{BUCKET_NAME}/", ""), "prediction.results"
)
batch_predict_results = download_gcs_files_as_json(
    gcs_files_prefix=batch_predict_output_files_prefix
)

# Display first two batch prediction results
for i, line in enumerate(batch_predict_results[:2]):
    embedding = np.array(line["prediction"]["embedding"])
    print(f"Embedding shape {i}: ", embedding.shape)

## Next steps

Explore the other [notebooks](https://github.com/google-health/medsiglip/blob/main/notebooks) to learn what else you can do with the model.

## Clean up resources

In [ ]:
# @markdown  Delete the experiment models and endpoints to recycle the resources
# @markdown  and avoid unnecessary continuous charges that may incur.

# Undeploy model and delete endpoint.
for endpoint in endpoints.values():
    endpoint.delete(force=True)

# Delete models.
for model in models.values():
    model.delete()

delete_bucket = False  # @param {type:"boolean"}
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_NAME